In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
import joblib
import os
import numpy as np

data=pd.read_csv('../data_store/final_data/final_data.csv')

In [2]:
data.head()

,timestamp,aqi,state,co,no,no2,o3,so2,pm2_5,pm10,nh3,temperature_2m,relative_humidity_2m,rain,wind_speed_10m,wind_direction_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm
0,2023-10-13 07:00:00,32.83,Alabama,198.60,0.0,1.05,56.51,0.13,2.84,3.31,0.19,16.929998,84.940186,0.0,7.594208,58.570484,18.279999,0.445
1,2023-10-13 08:00:00,31.95,Alabama,193.60,0.0,0.79,55.07,0.07,2.74,3.21,0.14,17.230000,84.153740,0.0,7.704336,52.594578,18.230000,0.445
2,2023-10-13 09:00:00,31.57,Alabama,191.93,0.0,0.67,53.64,0.09,2.81,3.30,0.11,17.529999,82.841130,0.0,8.287822,55.619600,18.279999,0.444
3,2023-10-13 10:00:00,31.25,Alabama,190.26,0.0,0.70,52.21,0.13,3.05,3.58,0.09,17.429998,83.905220,0.0,8.396570,59.036320,18.279999,0.443
4,2023-10-13 11:00:00,31.38,Alabama,191.93,0.0,0.79,50.78,0.17,3.35,3.92,0.08,17.380000,84.442380,0.0,7.787991,56.309900,18.230000,0.443


In [3]:
class AQIDataset(Dataset):
    def __init__(self, X, y, sequence_length):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y.reshape(-1, 1))  # Reshape y to match model output
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.X) - self.sequence_length + 1

    def __getitem__(self, idx):
        return (self.X[idx:idx + self.sequence_length], 
                self.y[idx + self.sequence_length - 1])

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
class HybridModel:
    def __init__(self, sequence_length=24, hidden_size=64, num_layers=2):
        self.sequence_length = sequence_length
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.lstm_model = None
        self.feature_scaler = MinMaxScaler()
        self.target_scaler = MinMaxScaler()
        
        self.order = (1, 1, 1)
        self.seasonal_order = (1, 1, 1, 24)
        self.sarimax_predictions = None
        
    def prepare_data(self, data):
        # Convert timestamp to datetime and sort
        data = data.copy()
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data.sort_values(['state', 'timestamp'])
        
        # Create features and target
        features = data.drop(['aqi', 'state', 'timestamp'], axis=1)
        target = data['aqi']
        
        # Split data
        train_size = int(len(data) * 0.8)
        train_features = features[:train_size]
        test_features = features[train_size:]
        train_target = target[:train_size]
        test_target = target[train_size:]
        
        # Scale data
        scaled_train_features = self.feature_scaler.fit_transform(train_features)
        scaled_test_features = self.feature_scaler.transform(test_features)
        scaled_train_target = self.target_scaler.fit_transform(train_target.values.reshape(-1, 1))
        scaled_test_target = self.target_scaler.transform(test_target.values.reshape(-1, 1))
        
        return (scaled_train_features, scaled_test_features, 
                scaled_train_target, scaled_test_target,
                train_target, test_target,
                data[:train_size], data[train_size:])
    
    def train_sarimax(self, train_data):
        # Group by state and train SARIMAX for each state
        residuals_list = []
        
        for state in train_data['state'].unique():
            state_data = train_data[train_data['state'] == state].copy()
            state_data = state_data.set_index('timestamp')
            state_data = state_data.asfreq('h')
            state_data = state_data.ffill().bfill()
            
            # Fit SARIMAX model
            model = SARIMAX(state_data['aqi'],
                          order=self.order,
                          seasonal_order=self.seasonal_order,
                          enforce_stationarity=False,
                          enforce_invertibility=False)
            
            try:
                fitted_model = model.fit(disp=False)
                predictions = fitted_model.get_prediction(start=0)
                state_residuals = state_data['aqi'] - predictions.predicted_mean
                residuals_list.append(state_residuals)
            except Exception as e:
                print(f"SARIMAX fitting error for state {state}: {str(e)}")
                # If SARIMAX fails, use simple differencing
                state_residuals = state_data['aqi'].diff().fillna(0)
                residuals_list.append(state_residuals)
        
        # Combine all residuals
        all_residuals = pd.concat(residuals_list)
        all_residuals = all_residuals.sort_index()
        
        return all_residuals.values
    
    def train_lstm(self, features, residuals, epochs=50, batch_size=32):
        dataset = AQIDataset(features, residuals, self.sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        if self.lstm_model is None:
            input_size = features.shape[1]
            self.lstm_model = LSTMModel(input_size, self.hidden_size, self.num_layers).to(self.device)
        
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.lstm_model.parameters())
        
        print("Training LSTM model...")
        self.lstm_model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch_X, batch_y in dataloader:
                batch_X = batch_X.to(self.device)
                batch_y = batch_y.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.lstm_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(dataloader):.4f}')
    
    def predict(self, features, data_subset):
        predictions_list = []
        
        for state in data_subset['state'].unique():
            state_mask = data_subset['state'] == state
            state_data = data_subset[state_mask].copy()
            state_data = state_data.set_index('timestamp')
            state_data = state_data.asfreq('h')
            state_data = state_data.ffill().bfill()
            
            try:
                # SARIMAX prediction
                model = SARIMAX(state_data['aqi'],
                              order=self.order,
                              seasonal_order=self.seasonal_order,
                              enforce_stationarity=False,
                              enforce_invertibility=False)
                fitted_model = model.fit(disp=False)
                sarimax_pred = fitted_model.get_prediction(start=0)
                sarimax_pred = sarimax_pred.predicted_mean.values.reshape(-1, 1)
            except Exception as e:
                print(f"SARIMAX prediction error for state {state}: {str(e)}")
                # Use simple moving average as fallback
                sarimax_pred = state_data['aqi'].rolling(window=24, min_periods=1).mean().values.reshape(-1, 1)
            
            # LSTM prediction
            state_features = features[state_mask]
            lstm_predictions = []
            
            self.lstm_model.eval()
            with torch.no_grad():
                for i in range(len(state_features) - self.sequence_length + 1):
                    sequence = torch.FloatTensor(
                        state_features[i:i + self.sequence_length]).unsqueeze(0).to(self.device)
                    pred = self.lstm_model(sequence)
                    lstm_predictions.append(pred.cpu().numpy())
            
            # Pad LSTM predictions
            pad_length = self.sequence_length - 1
            lstm_pred = np.pad(lstm_predictions, ((pad_length, 0), (0, 0)), mode='edge')
            
            # Combine predictions
            min_len = min(len(sarimax_pred), len(lstm_pred))
            state_predictions = sarimax_pred[:min_len] + lstm_pred[:min_len]
            predictions_list.append(state_predictions)
        
        # Combine all predictions
        all_predictions = np.vstack(predictions_list)
        return self.target_scaler.inverse_transform(all_predictions)

def train_and_evaluate_all_data(data):
    print("Initializing Hybrid Model...")
    hybrid_model = HybridModel()
    
    print("\nPreparing data...")
    (train_features, test_features, 
     train_target, test_target,
     original_train_target, original_test_target,
     train_data, test_data) = hybrid_model.prepare_data(data)
    
    print("\nTraining SARIMAX models and computing residuals...")
    residuals = hybrid_model.train_sarimax(train_data)
    
    print("\nTraining LSTM model...")
    hybrid_model.train_lstm(train_features, residuals)
    
    print("\nMaking predictions...")
    train_pred = hybrid_model.predict(train_features, train_data)
    test_pred = hybrid_model.predict(test_features, test_data)
    
    print("\nEvaluating model performance...")
    train_metrics = hybrid_model.evaluate(original_train_target.values, train_pred)
    test_metrics = hybrid_model.evaluate(original_test_target.values, test_pred)
    
    print("\nResults:")
    print(f"Train RMSE: {train_metrics['rmse']:.4f}")
    print(f"Test RMSE: {test_metrics['rmse']:.4f}")
    print(f"Train R2: {train_metrics['r2']:.4f}")
    print(f"Test R2: {test_metrics['r2']:.4f}")
    
    # Save model
    print("\nSaving model...")
    hybrid_model.save_model('models/combined_model')
    
    results = {
        'train_metrics': train_metrics,
        'test_metrics': test_metrics
    }
    
    return results, hybrid_model

# Usage example
results, model = train_and_evaluate_all_data(data)

Initializing Hybrid Model...

Preparing data...

Training SARIMAX models and computing residuals...


/home/arnabmitra/miniconda3/envs/PT_GPU/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/arnabmitra/miniconda3/envs/PT_GPU/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'



Training LSTM model...
Training LSTM model...
Epoch [10/50], Loss: 577.0893
Epoch [20/50], Loss: 576.7847
Epoch [30/50], Loss: 577.0734
Epoch [40/50], Loss: 574.5251
Epoch [50/50], Loss: 574.7218

Making predictions...


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (3,2)